In [1]:
import numpy as np
import pandas as pd
import zipfile
from zipfile import ZipFile
import glob
from openpyxl import load_workbook
pd.options.display.float_format = '{:,.2f}'.format

# PERIODO

In [2]:
start_date = '2024-01-01'
final_date = '2024-01-31'

# PARAMETRÍA COMISIONES

In [3]:
parametria_comisiones = pd.read_csv(r'G:\My Drive\RESPALDO-PC\AMEX\PARAMETRIA AMEX.csv',dtype={'meses_msi':object,
                                                                                              'comision_porcentaje':float})
parametria_comisiones['inicio'] = pd.to_datetime(parametria_comisiones['inicio'])
#parametria_comisiones['fin'] = pd.to_datetime(parametria_comisiones['fin'])
parametria_comisiones['comision_porcentaje_mdr'] = -0.0240
parametria_comisiones['comision_porcentaje_msi'] = -parametria_comisiones['comision_porcentaje']/100
parametria_comisiones.loc[parametria_comisiones['fuente'].str.contains('EPA'), "affiliation_type"] = 'AGGREGATOR'
parametria_comisiones.loc[parametria_comisiones['fuente'].str.contains('GRRCN'), "affiliation_type"] = 'OPTBLUE'
parametria_comisiones.loc[parametria_comisiones['meses_msi'].str.contains('99'),'meses_msi'] = '0'
parametria_comisiones.loc[parametria_comisiones['meses_msi'].str.contains('0'),'comision_porcentaje_msi'] = 0
parametria_comisiones['comision_porcentaje_final'] = (parametria_comisiones['comision_porcentaje_mdr'] +
                                                      parametria_comisiones['comision_porcentaje_msi'])
parametria_comisiones['index_comisiones'] = (parametria_comisiones['affiliation_type'].astype(str).replace(' ', '',regex=True)+'_'+
                                             parametria_comisiones['meses_msi'].astype(str).replace(' ', '',regex=True))

parametria_comisiones.head()

,motivo_del_cambio,naturaleza,status,inicio,fin,adquiriente,procesador,operativa,emisor,fuente,...,comision_monto,proceso,comentario,claves_comision,status_clip,comision_porcentaje_mdr,comision_porcentaje_msi,affiliation_type,comision_porcentaje_final,index_comisiones
0,Null,Fija,VIGENTE,2018-01-01,2999-12-31,AMEX,AMEX,TODAS,AMEX,EPA,...,0.00,Agregador,Costo de txn Amex Agregador General,NaN,1,-0.02,0.00,AGGREGATOR,-0.02,AGGREGATOR_0
1,NaN,TEMPORAL,TERMINADA,2020-04-15,2020-06-30,AMEX,AMEX,TODAS,AMEX,EPA,...,0.00,Agregador,Costo por Sobre tasa por MSI adicional a la tasa,NaN,1,-0.02,-0.03,AGGREGATOR,-0.05,AGGREGATOR_3
2,NaN,TEMPORAL,TERMINADA,2020-04-15,2020-06-30,AMEX,AMEX,TODAS,AMEX,EPA,...,0.00,Agregador,Costo por el uso de MSI adicional a la tasa y ...,NaN,1,-0.02,-0.00,AGGREGATOR,-0.03,AGGREGATOR_3
3,NaN,TEMPORAL,TERMINADA,2020-04-15,2020-06-30,AMEX,AMEX,TODAS,AMEX,EPA,...,0.00,Agregador,Costo por Sobre tasa por MSI adicional a la tasa,NaN,1,-0.02,-0.04,AGGREGATOR,-0.07,AGGREGATOR_6
4,NaN,TEMPORAL,TERMINADA,2020-04-15,2020-06-30,AMEX,AMEX,TODAS,AMEX,EPA,...,0.00,Agregador,Costo por el uso de MSI adicional a la tasa y ...,NaN,1,-0.02,-0.00,AGGREGATOR,-0.03,AGGREGATOR_6


In [4]:
parametria_comisiones.shape

(55, 29)

In [5]:
parametria_comisiones['fuente'].unique()

array(['EPA', 'GRRCN'], dtype=object)

In [6]:
parametria_comisiones['meses_msi'].unique()

array(['0', '3', '6', '9', '12', '18', '15'], dtype=object)

# COMISIONES EPA COBRADAS

In [7]:
comisiones_epa = parametria_comisiones[(parametria_comisiones['fuente'] == 'EPA') &
                                       ((parametria_comisiones['status'].str.contains('TERMINADA')) & (parametria_comisiones['comentario'].str.contains('Sobre tasa'))) |
                                       (parametria_comisiones['comentario'].str.contains('Agregador General'))]
comisiones_epa

,motivo_del_cambio,naturaleza,status,inicio,fin,adquiriente,procesador,operativa,emisor,fuente,...,comision_monto,proceso,comentario,claves_comision,status_clip,comision_porcentaje_mdr,comision_porcentaje_msi,affiliation_type,comision_porcentaje_final,index_comisiones
0,Null,Fija,VIGENTE,2018-01-01,2999-12-31,AMEX,AMEX,TODAS,AMEX,EPA,...,0.00,Agregador,Costo de txn Amex Agregador General,NaN,1,-0.02,0.00,AGGREGATOR,-0.02,AGGREGATOR_0
1,NaN,TEMPORAL,TERMINADA,2020-04-15,2020-06-30,AMEX,AMEX,TODAS,AMEX,EPA,...,0.00,Agregador,Costo por Sobre tasa por MSI adicional a la tasa,NaN,1,-0.02,-0.03,AGGREGATOR,-0.05,AGGREGATOR_3
3,NaN,TEMPORAL,TERMINADA,2020-04-15,2020-06-30,AMEX,AMEX,TODAS,AMEX,EPA,...,0.00,Agregador,Costo por Sobre tasa por MSI adicional a la tasa,NaN,1,-0.02,-0.04,AGGREGATOR,-0.07,AGGREGATOR_6
5,NaN,TEMPORAL,TERMINADA,2020-04-15,2020-06-30,AMEX,AMEX,TODAS,AMEX,EPA,...,0.00,Agregador,Costo por Sobre tasa por MSI adicional a la tasa,NaN,1,-0.02,-0.07,AGGREGATOR,-0.09,AGGREGATOR_9
7,NaN,TEMPORAL,TERMINADA,2020-04-15,2020-06-30,AMEX,AMEX,TODAS,AMEX,EPA,...,0.00,Agregador,Costo por Sobre tasa por MSI adicional a la tasa,NaN,1,-0.02,-0.08,AGGREGATOR,-0.11,AGGREGATOR_12
9,NaN,TEMPORAL,TERMINADA,2020-04-15,2020-06-30,AMEX,AMEX,TODAS,AMEX,EPA,...,0.00,Agregador,Costo por Sobre tasa por MSI adicional a la tasa,NaN,1,-0.02,-0.12,AGGREGATOR,-0.14,AGGREGATOR_18


In [8]:
comisiones_epa.shape

(6, 29)

# DB

In [9]:
# db_1 = pd.read_csv(r'G:\My Drive\RESPALDO-PC\AMEX\HISTORICO QUERIES\2021\AMEX SEPTIEMBRE 2021.csv'
#                    ,dtype={'status_code':object,'msi':object,'clave_autorizacion':object,'bin':object
#                            ,'last4':object,'order_id':object,'affiliation':object,'sub_affiliation':object
#                            ,'mcc':object})
# db_2 = pd.read_csv(r'G:\My Drive\RESPALDO-PC\AMEX\HISTORICO QUERIES\2021\AMEX AGOSTO 2021.csv'
#                    ,dtype={'status_code':object,'msi':object,'clave_autorizacion':object,'bin':object
#                            ,'last4':object,'order_id':object,'affiliation':object,'sub_affiliation':object
#                            ,'mcc':object})
db_1 = pd.read_csv(r'G:\My Drive\RESPALDO-PC\AMEX\HISTORICO QUERIES\2024\AMEX ENERO 2024.csv'
                   ,dtype={'STATUS_CODE':object,'MSI':object,'CLAVE_AUTORIZACION':object,'BIN':object
                           ,'LAST4':object,'ORDER_ID':object,'AFFILIATION':object,'SUB_AFFILIATION':object
                           ,'MCC':object})
# db_2 = pd.read_csv(r'G:\My Drive\RESPALDO-PC\AMEX\HISTORICO QUERIES\2021\QUERIES CONSOLIDADO\AMEX OCTUBRE 2021.csv'
#                    ,dtype={'STATUS_CODE':object,'MSI':object,'CLAVE_AUTORIZACION':object,'BIN':object
#                            ,'LAST4':object,'ORDER_ID':object,'AFFILIATION':object,'SUB_AFFILIATION':object
#                            ,'MCC':object})
# db_3 = pd.read_csv(r'G:\My Drive\RESPALDO-PC\AMEX\HISTORICO QUERIES\2021\QUERIES CONSOLIDADO\AMEX JUNIO-JULIO 2021.csv'
#                    ,dtype={'STATUS_CODE':object,'MSI':object,'CLAVE_AUTORIZACION':object,'BIN':object
#                            ,'LAST4':object,'ORDER_ID':object,'AFFILIATION':object,'SUB_AFFILIATION':object
#                            ,'MCC':object})
# db_4 = pd.read_csv(r'G:\My Drive\RESPALDO-PC\AMEX\HISTORICO QUERIES\2021\QUERIES CONSOLIDADO\AMEX AGOSTO-SEPTIEMBRE 2021.csv'
#                    ,dtype={'STATUS_CODE':object,'MSI':object,'CLAVE_AUTORIZACION':object,'BIN':object
#                            ,'LAST4':object,'ORDER_ID':object,'AFFILIATION':object,'SUB_AFFILIATION':object
#                            ,'MCC':object})

In [10]:
db = pd.concat([db_1])
db.columns = db.columns.str.strip().str.lower() 
db['order_id'] = db['order_id'].str.strip()
db['order_id'] = db['order_id'].str.upper()
db['date_transaction'] = pd.to_datetime(db['date_transaction'])
db['INDEX_TOTAL'] = (db['bin'].astype(str).replace(' ', '',regex=True)+'_'
                       +db['last4'].astype(str).replace(' ', '',regex=True)+'_'
                       +db['order_id'].astype(str).replace(' ', '',regex=True)+'_'
                       +db['amount'].astype(str).replace(' ', '',regex=True))
db['INDEX_PARCIAL'] = (db['bin'].astype(str).replace(' ', '',regex=True)+'_'
                       +db['last4'].astype(str).replace(' ', '',regex=True)+'_'
                       +db['clave_autorizacion'].astype(str).replace(' ', '',regex=True)+'_'
                       +db['amount'].astype(str).replace(' ', '',regex=True))
db['index_comisiones'] = (db['affiliation_type'].astype(str).replace(' ', '',regex=True)+'_'
                       +db['msi'].str[:-6].astype(str).replace(' ', '',regex=True))
db['order_id'].fillna('SIN ORDER_ID EN DB',inplace=True)
db.head()

,merchant_id,transaction_id,order_id,clave_autorizacion,affiliation,affiliation_type,sub_affiliation,mcc,date_transaction,created_at,...,decline_code,msi,device_id,clip_reader,status_msg,issuer,brand,INDEX_TOTAL,INDEX_PARCIAL,index_comisiones
0,c090de31-388f-41c9-9bf0-5ae7adf17975,5725dcb9-4054-4cb0-94ff-a9be5b970b4b,S0CKBKMHDQPP,010339200576064,8138464136,OPTBLUE,710910,5812,2023-12-25,2023-12-25T06:05:50Z,...,NaN,0.00000,P5220314003479,Clip Classic,Paid,AMEX,American Express,379794_1004_S0CKBKMHDQPP_380.0,379794_1004_010339200576064_380.0,OPTBLUE_0
1,4a9c6399-ae89-4b68-9607-cbd05f43e110,dcac4d6b-cd68-40c6-af7f-1999753fe76c,Z5MDOPDNOIPD,010339208448068,8138464136,OPTBLUE,1908592,5813,2023-12-25,2023-12-25T06:06:06Z,...,NaN,0.00000,N500W4P2184,Clip Total,Paid,AMEX,American Express,371292_3000_Z5MDOPDNOIPD_3210.0,371292_3000_010339208448068_3210.0,OPTBLUE_0
2,8c22ed4b-9555-4018-8fcb-c05df672c953,54d07a23-3169-44ac-b3a1-44d6b781fe70,HZJNZXXXDMRT,010339220050066,8138464136,OPTBLUE,1658106,5813,2023-12-25,2023-12-25T06:06:58Z,...,NaN,0.00000,N500W4M8642,Clip Total,Paid,AMEX,AMERICAN EXPRESS,376667_2037_HZJNZXXXDMRT_200.0,376667_2037_010339220050066_200.0,OPTBLUE_0
3,3b5049b8-3217-49ca-86ea-caaaa3ec3308,c90ef4ae-3349-4a07-af68-6fb916b8e67a,5IT5LIBALHVF,010339219163060,8138464136,OPTBLUE,1106905,5813,2023-12-25,2023-12-25T06:07:45Z,...,NaN,0.00000,N500W4E2326,Clip Total,Paid,AMEX,American Express,377978_1006_5IT5LIBALHVF_180.0,377978_1006_010339219163060_180.0,OPTBLUE_0
4,8c22ed4b-9555-4018-8fcb-c05df672c953,548bd5a9-596a-4803-86c2-6e0d95f7e2cb,MMX2JP07EXV1,010339204880066,8138464136,OPTBLUE,1658106,5813,2023-12-25,2023-12-25T06:08:21Z,...,NaN,0.00000,N500W4M8642,Clip Total,Paid,AMEX,AMERICAN EXPRESS,376718_1042_MMX2JP07EXV1_200.0,376718_1042_010339204880066_200.0,OPTBLUE_0


In [11]:
db.shape

(337597, 27)

In [12]:
db['msi'].unique()

array(['0.00000', '3.00000', '9.00000', '6.00000', '12.00000', '24.00000',
       '18.00000'], dtype=object)

In [13]:
db['index_comisiones'].unique()

array(['OPTBLUE_0', 'OPTBLUE_3', 'OPTBLUE_9', 'nan_0', 'OPTBLUE_6',
       'nan_3', 'OPTBLUE_12', 'nan_24', 'nan_12', 'nan_6', 'OPTBLUE_18',
       'AGGREGATOR_0', 'OPTBLUE_24', 'AGGREGATOR_6', 'nan_9', 'nan_18',
       'AGGREGATOR_9', 'CARD_PRESENT_0', 'ECOMMERCE_0'], dtype=object)

In [14]:
db.dtypes

merchant_id                   object
transaction_id                object
order_id                      object
clave_autorizacion            object
affiliation                   object
affiliation_type              object
sub_affiliation               object
mcc                           object
date_transaction      datetime64[ns]
created_at                    object
updated_at                    object
psp                           object
amount                       float64
bin                           object
receipt_no                    object
last4                         object
status_code                   object
decline_code                 float64
msi                           object
device_id                     object
clip_reader                   object
status_msg                    object
issuer                        object
brand                         object
INDEX_TOTAL                   object
INDEX_PARCIAL                 object
index_comisiones              object
d

In [15]:
duplicates = db[db.duplicated(['transaction_id'] , keep=False)]
duplicates.head()

,merchant_id,transaction_id,order_id,clave_autorizacion,affiliation,affiliation_type,sub_affiliation,mcc,date_transaction,created_at,...,decline_code,msi,device_id,clip_reader,status_msg,issuer,brand,INDEX_TOTAL,INDEX_PARCIAL,index_comisiones
321803,62ab9ad3-50e4-4c56-b7fe-4b0d6691b426,40f02f7b-abd8-4a35-bb8a-7dfdf4490405,OC2WO7ERVMNY,NaN,8138464144,OPTBLUE,2015017,7399,2024-01-30,2024-01-30T17:08:54Z,...,NaN,0.00000,80010001,Clip Classic,Deny,AMEX,AMERICAN EXPRESS,371776_2007_OC2WO7ERVMNY_17214.33,371776_2007_nan_17214.33,OPTBLUE_0
321804,62ab9ad3-50e4-4c56-b7fe-4b0d6691b426,40f02f7b-abd8-4a35-bb8a-7dfdf4490405,OC2WO7ERVMNY,NaN,8138464144,ECOMMERCE,2015017,7399,2024-01-30,2024-01-30T17:08:54Z,...,NaN,0.00000,80010001,Clip Classic,Deny,AMEX,AMERICAN EXPRESS,371776_2007_OC2WO7ERVMNY_17214.33,371776_2007_nan_17214.33,ECOMMERCE_0
321864,ceb5c5ea-1735-4745-9389-43122cf0b98e,5d6be542-93bf-4498-a05a-9c39f1e4cec0,OLUXGABGTF4U,010901004781065,8138464227,ECOMMERCE,9686,5499,2024-01-30,2024-01-30T17:14:53Z,...,NaN,0.00000,80010001,Clip Classic,Paid,AMEXBANK,AMERICAN EXPRESS,371690_7005_OLUXGABGTF4U_36103.89,371690_7005_010901004781065_36103.89,ECOMMERCE_0
321865,ceb5c5ea-1735-4745-9389-43122cf0b98e,5d6be542-93bf-4498-a05a-9c39f1e4cec0,OLUXGABGTF4U,010901004781065,8138464227,OPTBLUE,9686,5499,2024-01-30,2024-01-30T17:14:53Z,...,NaN,0.00000,80010001,Clip Classic,Paid,AMEXBANK,AMERICAN EXPRESS,371690_7005_OLUXGABGTF4U_36103.89,371690_7005_010901004781065_36103.89,OPTBLUE_0


In [16]:
duplicates.shape

(4, 27)

# DB CON COMISIONES

In [17]:
db_final = db.merge(comisiones_epa[['index_comisiones','comision_porcentaje_mdr',
                                   'comision_porcentaje_msi','comision_porcentaje_final']],on='index_comisiones',how='left')
db_final['comision_porcentaje_final'].fillna(0,inplace=True)
db_final['comision_mdr_db'] = db_final['amount'] * db_final['comision_porcentaje_mdr']
db_final['iva_comision_mdr_db'] = db_final['comision_mdr_db'] * 0.1600
db_final['comision_msi_db'] = db_final['amount'] * db_final['comision_porcentaje_msi']
db_final['iva_comision_msi_db'] = db_final['comision_msi_db'] * 0.1600
db_final.head()

,merchant_id,transaction_id,order_id,clave_autorizacion,affiliation,affiliation_type,sub_affiliation,mcc,date_transaction,created_at,...,INDEX_TOTAL,INDEX_PARCIAL,index_comisiones,comision_porcentaje_mdr,comision_porcentaje_msi,comision_porcentaje_final,comision_mdr_db,iva_comision_mdr_db,comision_msi_db,iva_comision_msi_db
0,c090de31-388f-41c9-9bf0-5ae7adf17975,5725dcb9-4054-4cb0-94ff-a9be5b970b4b,S0CKBKMHDQPP,010339200576064,8138464136,OPTBLUE,710910,5812,2023-12-25,2023-12-25T06:05:50Z,...,379794_1004_S0CKBKMHDQPP_380.0,379794_1004_010339200576064_380.0,OPTBLUE_0,NaN,NaN,0.00,NaN,NaN,NaN,NaN
1,4a9c6399-ae89-4b68-9607-cbd05f43e110,dcac4d6b-cd68-40c6-af7f-1999753fe76c,Z5MDOPDNOIPD,010339208448068,8138464136,OPTBLUE,1908592,5813,2023-12-25,2023-12-25T06:06:06Z,...,371292_3000_Z5MDOPDNOIPD_3210.0,371292_3000_010339208448068_3210.0,OPTBLUE_0,NaN,NaN,0.00,NaN,NaN,NaN,NaN
2,8c22ed4b-9555-4018-8fcb-c05df672c953,54d07a23-3169-44ac-b3a1-44d6b781fe70,HZJNZXXXDMRT,010339220050066,8138464136,OPTBLUE,1658106,5813,2023-12-25,2023-12-25T06:06:58Z,...,376667_2037_HZJNZXXXDMRT_200.0,376667_2037_010339220050066_200.0,OPTBLUE_0,NaN,NaN,0.00,NaN,NaN,NaN,NaN
3,3b5049b8-3217-49ca-86ea-caaaa3ec3308,c90ef4ae-3349-4a07-af68-6fb916b8e67a,5IT5LIBALHVF,010339219163060,8138464136,OPTBLUE,1106905,5813,2023-12-25,2023-12-25T06:07:45Z,...,377978_1006_5IT5LIBALHVF_180.0,377978_1006_010339219163060_180.0,OPTBLUE_0,NaN,NaN,0.00,NaN,NaN,NaN,NaN
4,8c22ed4b-9555-4018-8fcb-c05df672c953,548bd5a9-596a-4803-86c2-6e0d95f7e2cb,MMX2JP07EXV1,010339204880066,8138464136,OPTBLUE,1658106,5813,2023-12-25,2023-12-25T06:08:21Z,...,376718_1042_MMX2JP07EXV1_200.0,376718_1042_010339204880066_200.0,OPTBLUE_0,NaN,NaN,0.00,NaN,NaN,NaN,NaN


In [18]:
db_final.shape

(337597, 34)

# EPA

In [19]:
allfiles = glob.glob(r'G:\.shortcut-targets-by-id\1_Ma5aBDa2iehbpuQ3dnO1tUN_wp4Jgka\EPA\EPA files\2024 EPA Amex\**\*.txt', recursive=True)
allfiles

['G:\\.shortcut-targets-by-id\\1_Ma5aBDa2iehbpuQ3dnO1tUN_wp4Jgka\\EPA\\EPA files\\2024 EPA Amex\\01. ENERO\\2024-01-15..txt']

In [20]:
names_header=["1","2","3","4","5","6","7","8","9","10","A","12","13","14","15","16","17","18","19","order_id",
              "21","22","23","24","25","26","27","28"]

In [21]:
epa = pd.concat([pd.read_csv(f,names=names_header, sep=',',dtype=object) for f in allfiles])
epa['order_id'] = epa['order_id'].str.strip()
epa.head()

,1,2,3,4,5,6,7,8,9,10,...,19,order_id,21,22,23,24,25,26,27,28
0,9351789434,00010101,000000,0000000000,00000,0,0,20240115,004703,002608,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9351789434,20240116,055586,0000000000,00000,1,0,92355,072,000000,...,92355,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9351789434,20240116,055586,9351789434,00001,3,0,20240113,935178943440114,95000,...,0,46687159,20240122,20240112,6,000,92355,95000,000,NaN
3,9351789434,20240116,055586,9351789434,00001,4,0,20240113,0,805,...,000LMNRTYDPHHMY,LMNRTYDPHHMY,000LMNRTYDPHHMY,000,000,NaN,NaN,NaN,NaN,NaN
4,9351789434,99991231,999999,9999999999,99999,9,0,20240115,004703,002608,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
epa.shape

(5, 28)

In [23]:
epa['6'].unique()

array(['0', '1', '3', '4', '9'], dtype=object)

In [24]:
epa_0 = epa[(epa['6'] == '0')]
epa_1 = epa[(epa['6'] == '1')]
epa_3 = epa[(epa['6'] == '3')]
epa_4 = epa[(epa['6'] == '4')]
epa_5 = epa[(epa['6'] == '5')]
epa_9 = epa[(epa['6'] == '9')]

## EPA 0

In [25]:
epa_0.head()

,1,2,3,4,5,6,7,8,9,10,...,19,order_id,21,22,23,24,25,26,27,28
0,9351789434,00010101,000000,0000000000,00000,0,0,20240115,004703,002608,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
epa_0.shape

(1, 28)

## EPA 1

In [27]:
epa_1['2'] = pd.to_datetime(epa_1['2'], format='%Y%m%d').dt.date
epa_1['2'] = pd.to_datetime(epa_1['2'])
epa_1.sort_values(by=['2'],inplace=True)
epa_1.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\2201093678.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  epa_1['2'] = pd.to_datetime(epa_1['2'], format='%Y%m%d').dt.date
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\2201093678.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  epa_1['2'] = pd.to_datetime(epa_1['2'])
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\2201093678.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

,1,2,3,4,5,6,7,8,9,10,...,19,order_id,21,22,23,24,25,26,27,28
1,9351789434,2024-01-16,055586,0000000000,00000,1,0,92355,072,000000,...,92355,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
epa_1.shape

(1, 28)

In [29]:
epa_1_final = epa_1[['6','2','8','15','16','17','18','19']]
epa_1_final.head()

,6,2,8,15,16,17,18,19
1,1,2024-01-16,92355,95000,-2280,-365,000,92355


In [30]:
epa_1_final['8'] = epa_1_final['8'].astype(float)
epa_1_final['15'] = epa_1_final['15'].astype(float)
epa_1_final['16'] = epa_1_final['16'].astype(float)
epa_1_final['17'] = epa_1_final['17'].astype(float)
epa_1_final['18'] = epa_1_final['18'].astype(float)
epa_1_final['19'] = epa_1_final['19'].astype(float)

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\127189751.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  epa_1_final['8'] = epa_1_final['8'].astype(float)
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\127189751.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  epa_1_final['15'] = epa_1_final['15'].astype(float)
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\127189751.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

In [31]:
epa_1_final.rename(columns ={'6':'Type','2':'Payment_date'},inplace=True)

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\739277314.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  epa_1_final.rename(columns ={'6':'Type','2':'Payment_date'},inplace=True)


In [32]:
epa_1_final['Payment_amount'] = epa_1_final['8']/100
epa_1_final['Gross_amount'] = epa_1_final['15']/100
epa_1_final['Discount_amount'] = epa_1_final['16']/100
epa_1_final['Taxes_amount'] = epa_1_final['17']/100
epa_1_final['Discount_aceleration_amount'] = epa_1_final['18']/100
epa_1_final['Net_amount'] = epa_1_final['19']/100
epa_1_final.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\2918445751.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  epa_1_final['Payment_amount'] = epa_1_final['8']/100
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\2918445751.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  epa_1_final['Gross_amount'] = epa_1_final['15']/100
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\2918445751.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

,Type,Payment_date,8,15,16,17,18,19,Payment_amount,Gross_amount,Discount_amount,Taxes_amount,Discount_aceleration_amount,Net_amount
1,1,2024-01-16,"92,355.00","95,000.00","-2,280.00",-365.00,0.00,"92,355.00",923.55,950.00,-22.80,-3.65,0.00,923.55


In [33]:
epa_1_excel = epa_1_final[(epa_1_final['Payment_date'] >= start_date) & (epa_1_final['Payment_date'] <= final_date)]

In [34]:
epa_1_excel_final = epa_1_excel.groupby(['Payment_date']).agg({'Payment_amount':sum,'Gross_amount':sum,'Discount_amount':sum
                                                        ,'Taxes_amount':sum,'Discount_aceleration_amount':sum
                                                        ,'Net_amount':sum}).reset_index()
epa_1_excel_final.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\2945002399.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  epa_1_excel_final = epa_1_excel.groupby(['Payment_date']).agg({'Payment_amount':sum,'Gross_amount':sum,'Discount_amount':sum


,Payment_date,Payment_amount,Gross_amount,Discount_amount,Taxes_amount,Discount_aceleration_amount,Net_amount
0,2024-01-16,923.55,950.00,-22.80,-3.65,0.00,923.55


In [35]:
epa_1_excel_final.shape

(1, 7)

In [36]:
epa_1_excel_final.to_excel('EPA_1.xlsx')

## EPA 3

In [37]:
epa_3['2'] = pd.to_datetime(epa_3['2'], format='%Y%m%d').dt.date
epa_3['2'] = pd.to_datetime(epa_3['2'])
epa_3.sort_values(by=['2'],inplace=True)
epa_3.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\1828633791.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  epa_3['2'] = pd.to_datetime(epa_3['2'], format='%Y%m%d').dt.date
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\1828633791.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  epa_3['2'] = pd.to_datetime(epa_3['2'])
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\1828633791.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

,1,2,3,4,5,6,7,8,9,10,...,19,order_id,21,22,23,24,25,26,27,28
2,9351789434,2024-01-16,055586,9351789434,00001,3,0,20240113,935178943440114,95000,...,0,46687159,20240122,20240112,6,000,92355,95000,000,NaN


In [38]:
epa_3.shape

(1, 28)

In [39]:
epa_3.to_csv('EPA_3.csv')

In [40]:
epa_3_final = epa_3[['6','2','10','A','12','13','14','15','16','25','26','27','28']]
epa_3_final.reset_index(drop=True, inplace=True)
epa_3_final.head()

,6,2,10,A,12,13,14,15,16,25,26,27,28
0,3,2024-01-16,95000,95000,-2280,-365,000,92355,1,92355,95000,000,NaN


In [41]:
epa_3_final['10'] = epa_3_final['10'].astype(float)
epa_3_final['A'] = epa_3_final['A'].astype(float)
epa_3_final['12'] = epa_3_final['12'].astype(float)
epa_3_final['13'] = epa_3_final['13'].astype(float)
epa_3_final['14'] = epa_3_final['14'].astype(float)
epa_3_final['15'] = epa_3_final['15'].astype(float)
epa_3_final['16'] = epa_3_final['16'].astype(float)
epa_3_final['25'] = epa_3_final['25'].astype(float)
epa_3_final['26'] = epa_3_final['26'].astype(float)
epa_3_final['27'] = epa_3_final['27'].astype(float)
epa_3_final['28'] = epa_3_final['28'].astype(float)

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\2821721407.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  epa_3_final['10'] = epa_3_final['10'].astype(float)
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\2821721407.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  epa_3_final['A'] = epa_3_final['A'].astype(float)
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\2821721407.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

In [42]:
epa_3_final.rename(columns ={'6':'Type','2':'Payment_date','16':'Register_number'},inplace=True)

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\10605692.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  epa_3_final.rename(columns ={'6':'Type','2':'Payment_date','16':'Register_number'},inplace=True)


In [43]:
epa_3_final['Gross_amount'] = epa_3_final['10']/100
epa_3_final['Gross_amount_amex'] = epa_3_final['A']/100
epa_3_final['Discount_amount'] = epa_3_final['12']/100
epa_3_final['Taxes_amount'] = epa_3_final['13']/100
epa_3_final['Discount_aceleration_amount'] = epa_3_final['14']/100
epa_3_final['Net_amount'] = epa_3_final['15']/100
epa_3_final['Aceleration_net_amount'] = epa_3_final['25']/100
epa_3_final['Gross_amount_trans_cambio'] = epa_3_final['26']/100
epa_3_final['Credit_gross_amount'] = epa_3_final['27']/100
epa_3_final['Unknown'] = epa_3_final['28']/100
epa_3_final.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\1787877787.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  epa_3_final['Gross_amount'] = epa_3_final['10']/100
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\1787877787.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  epa_3_final['Gross_amount_amex'] = epa_3_final['A']/100
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\1787877787.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

,Type,Payment_date,10,A,12,13,14,15,Register_number,25,...,Gross_amount,Gross_amount_amex,Discount_amount,Taxes_amount,Discount_aceleration_amount,Net_amount,Aceleration_net_amount,Gross_amount_trans_cambio,Credit_gross_amount,Unknown
0,3,2024-01-16,"95,000.00","95,000.00","-2,280.00",-365.00,0.00,"92,355.00",1.00,"92,355.00",...,950.00,950.00,-22.80,-3.65,0.00,923.55,923.55,950.00,0.00,NaN


In [44]:
epa_3_excel = epa_3_final[(epa_3_final['Payment_date'] >= start_date) & (epa_3_final['Payment_date'] <= final_date)]

In [45]:
epa_3_excel_final = epa_3_excel.groupby(['Payment_date']).agg({'Register_number':sum,'Gross_amount':sum,'Gross_amount_amex':sum
                                                               ,'Discount_amount':sum,'Taxes_amount':sum
                                                               ,'Discount_aceleration_amount':sum,'Net_amount':sum
                                                               ,'Aceleration_net_amount':sum
                                                               ,'Gross_amount_trans_cambio':sum,'Credit_gross_amount':sum
                                                               ,'Unknown':sum}).reset_index()
epa_3_excel_final.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\2673989929.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  epa_3_excel_final = epa_3_excel.groupby(['Payment_date']).agg({'Register_number':sum,'Gross_amount':sum,'Gross_amount_amex':sum


,Payment_date,Register_number,Gross_amount,Gross_amount_amex,Discount_amount,Taxes_amount,Discount_aceleration_amount,Net_amount,Aceleration_net_amount,Gross_amount_trans_cambio,Credit_gross_amount,Unknown
0,2024-01-16,1.00,950.00,950.00,-22.80,-3.65,0.00,923.55,923.55,950.00,0.00,0.00


In [46]:
epa_3_excel_final.shape

(1, 12)

In [47]:
epa_3_excel_final.to_csv('EPA_3.csv')

## EPA 4

In [48]:
epa_4['8'] = pd.to_datetime(epa_4['8'], format='%Y%m%d').dt.date
epa_4['2'] = pd.to_datetime(epa_4['2'], format='%Y%m%d').dt.date

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\424667492.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  epa_4['8'] = pd.to_datetime(epa_4['8'], format='%Y%m%d').dt.date
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\424667492.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  epa_4['2'] = pd.to_datetime(epa_4['2'], format='%Y%m%d').dt.date


In [49]:
epa_4['8'] = pd.to_datetime(epa_4['8'])
epa_4['2'] = pd.to_datetime(epa_4['2'])

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\1129604653.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  epa_4['8'] = pd.to_datetime(epa_4['8'])
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\1129604653.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  epa_4['2'] = pd.to_datetime(epa_4['2'])


In [50]:
epa_4.head()

,1,2,3,4,5,6,7,8,9,10,...,19,order_id,21,22,23,24,25,26,27,28
3,9351789434,2024-01-16,055586,9351789434,00001,4,0,2024-01-13,0,805,...,000LMNRTYDPHHMY,LMNRTYDPHHMY,000LMNRTYDPHHMY,000,000,NaN,NaN,NaN,NaN,NaN


In [51]:
epa_4.shape

(1, 28)

In [52]:
epa_4.to_csv('EPA_4.csv',index=False)

In [53]:
txn_epa = epa_4[['order_id','6','8','A','12','22','23','2','15','17']]
txn_epa["12"] = txn_epa['12'].astype(float)
txn_epa["22"] = txn_epa['22'].astype(float)
txn_epa["23"] = txn_epa['23'].astype(float)


C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\3567835031.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  txn_epa["12"] = txn_epa['12'].astype(float)
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\3567835031.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  txn_epa["22"] = txn_epa['22'].astype(float)
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\3567835031.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

In [54]:
txn_epa['total_amount_bruto'] = txn_epa['12']/100
txn_epa['fee_amount_mdr'] = -(txn_epa['total_amount_bruto']*0.0240)
txn_epa['fee_amount_msi'] = txn_epa['22']/100
txn_epa['iva_msi'] = txn_epa['fee_amount_msi']*0.1600
txn_epa['iva_mdr'] = txn_epa['fee_amount_mdr']*0.1600
txn_epa['bin_bank'] = txn_epa['A'].str[:6].astype(str)
txn_epa['last4_bank'] = txn_epa['A'].str[-8:-4].astype(str)

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\1656383301.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  txn_epa['total_amount_bruto'] = txn_epa['12']/100
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\1656383301.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  txn_epa['fee_amount_mdr'] = -(txn_epa['total_amount_bruto']*0.0240)
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\1656383301.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [55]:
txn_epa.rename(columns ={'6':'Type','8':'Date_process_roc','12':'Total_amount_charge_roc',
                         '22':'Svc_fee_amount_roc','23':'Svc_fee_amount_accel_roc','2':'roc_deposit_date',
                        '17':'rechazo_amex'},inplace=True)
txn_epa['INDEX_TOTAL'] = (txn_epa['bin_bank'].astype(str).replace(' ', '',regex=True)+'_'
                            +txn_epa['last4_bank'].astype(str).replace(' ', '',regex=True)+'_'
                            +txn_epa['order_id'].astype(str).replace(' ', '',regex=True)+'_'
                            +txn_epa['total_amount_bruto'].astype(str).replace(' ', '',regex=True))
# txn_epa['INDEX_PARCIAL'] = (txn_epa['bin_bank'].astype(str).replace(' ', '',regex=True)+'_'
#                             +txn_epa['last4_bank'].astype(str).replace(' ', '',regex=True)+'_'
#                             +txn_epa['codigo_aprobacion'].astype(str).replace(' ', '',regex=True)+'_'
#                             +txn_epa['total_amount_bruto'].astype(str).replace(' ', '',regex=True))
txn_epa.fillna('SIN ORDER_ID EN EPA',inplace=True)
txn_epa.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\3748546502.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  txn_epa.rename(columns ={'6':'Type','8':'Date_process_roc','12':'Total_amount_charge_roc',
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\3748546502.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  txn_epa['INDEX_TOTAL'] = (txn_epa['bin_bank'].astype(str).replace(' ', '',regex=True)+'_'
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\3748546502.py:12: SettingWithCopyWarning: 
A value is trying to be set on a cop

,order_id,Type,Date_process_roc,A,Total_amount_charge_roc,Svc_fee_amount_roc,Svc_fee_amount_accel_roc,roc_deposit_date,15,rechazo_amex,total_amount_bruto,fee_amount_mdr,fee_amount_msi,iva_msi,iva_mdr,bin_bank,last4_bank,INDEX_TOTAL
3,LMNRTYDPHHMY,4,2024-01-13,376669*****2016****,"95,000.00",0.00,0.00,2024-01-16,0,00,950.00,-22.80,0.00,0.00,-3.65,376669,2016,376669_2016_LMNRTYDPHHMY_950.0


In [56]:
txn_epa.shape

(1, 18)

In [57]:
txn_epa['rechazo_amex'].unique()

array(['00'], dtype=object)

In [58]:
# txn_epa.to_excel('EPA CONSOLIDADO.xlsx',index=False)

### DATA PARA PROVISION

In [59]:
start_date_prov = '2023-12-20'

In [60]:
txn_epa_prov = txn_epa[(txn_epa['Date_process_roc'] >= start_date_prov)]
txn_epa_prov.head()

,order_id,Type,Date_process_roc,A,Total_amount_charge_roc,Svc_fee_amount_roc,Svc_fee_amount_accel_roc,roc_deposit_date,15,rechazo_amex,total_amount_bruto,fee_amount_mdr,fee_amount_msi,iva_msi,iva_mdr,bin_bank,last4_bank,INDEX_TOTAL
3,LMNRTYDPHHMY,4,2024-01-13,376669*****2016****,"95,000.00",0.00,0.00,2024-01-16,0,00,950.00,-22.80,0.00,0.00,-3.65,376669,2016,376669_2016_LMNRTYDPHHMY_950.0


In [61]:
txn_epa_prov.shape

(1, 18)

In [62]:
txn_epa_prov.to_csv('AMEX PROVISION EPA ENERO 2024.csv',index=False)

## EPA 5

In [63]:
epa_5['2'] = pd.to_datetime(epa_5['2'], format='%Y%m%d').dt.date

In [64]:
epa_5['2'] = pd.to_datetime(epa_5['2'])

In [65]:
epa_5.head()

,1,2,3,4,5,6,7,8,9,10,...,19,order_id,21,22,23,24,25,26,27,28


In [66]:
epa_5.shape

(0, 28)

In [67]:
txn_epa_adj = epa_5[['2','8','9','10','A','12','13','14','15','16','18']]
txn_epa_adj.rename(columns ={'2':'roc_deposit_date','8':'clave_autorizacion','14':'no_tarjeta','15':'adj_id','16':'motivo_adj'},inplace=True) 
txn_epa_adj["9"] = txn_epa_adj['9'].astype(float)
txn_epa_adj["10"] = txn_epa_adj['10'].astype(float)
txn_epa_adj["A"] = txn_epa_adj['A'].astype(float)
txn_epa_adj["12"] = txn_epa_adj['12'].astype(float)
txn_epa_adj["13"] = txn_epa_adj['13'].astype(float)


C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\4211780772.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  txn_epa_adj.rename(columns ={'2':'roc_deposit_date','8':'clave_autorizacion','14':'no_tarjeta','15':'adj_id','16':'motivo_adj'},inplace=True)


In [68]:
txn_epa_adj['importe_adj_bruto'] = txn_epa_adj['9']/100
txn_epa_adj['tasa_adj'] = txn_epa_adj['10']/100
txn_epa_adj['importe_impuesto'] = txn_epa_adj['A']/100
txn_epa_adj['tasa_servicio_plan_n'] = txn_epa_adj['12']/100
txn_epa_adj['importe_adj_neto'] = txn_epa_adj['13']/100
txn_epa_adj['bin_bank'] = txn_epa_adj['no_tarjeta'].str[:6].astype(str)
txn_epa_adj['last4_bank'] = txn_epa_adj['no_tarjeta'].str[-8:-4].astype(str)

In [69]:
txn_epa_adj.head()

,roc_deposit_date,clave_autorizacion,9,10,A,12,13,no_tarjeta,adj_id,motivo_adj,18,importe_adj_bruto,tasa_adj,importe_impuesto,tasa_servicio_plan_n,importe_adj_neto,bin_bank,last4_bank


In [70]:
txn_epa_adj.shape

(0, 18)

In [71]:
txn_epa_adj.to_excel(r'ADJ CONSOLIDADO.xlsx',index=False)

In [72]:
txn_epa_adj_final = txn_epa_adj[(txn_epa_adj['roc_deposit_date'] >= start_date) & (txn_epa_adj['roc_deposit_date'] <= final_date)]

## EPA 9

In [73]:
epa_9.head()

,1,2,3,4,5,6,7,8,9,10,...,19,order_id,21,22,23,24,25,26,27,28
4,9351789434,99991231,999999,9999999999,99999,9,0,20240115,004703,002608,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
epa_9.shape

(1, 28)

# CONCILIACION ALL

In [75]:
cruce_1_previo = txn_epa.merge(db_final[['order_id','transaction_id','date_transaction','amount','status_code','msi','bin',
                                         'last4','clave_autorizacion','comision_porcentaje_mdr','comision_porcentaje_msi',
                                         'comision_porcentaje_final','comision_mdr_db','iva_comision_mdr_db',
                                         'comision_msi_db','iva_comision_msi_db']],on='order_id',how='left')
cruce_1_previo[['transaction_id','status_code','msi']].fillna('NO ENCONTRADO',inplace=True)
cruce_1_previo['amount'].fillna(0,inplace=True)
cruce_1_previo['status_code'].fillna('NO ENCONTRADO',inplace=True)
cruce_1_previo.loc[~cruce_1_previo['status_code'].str.contains('3'),'amount_new'] = cruce_1_previo['amount']
cruce_1_previo.loc[cruce_1_previo['status_code'].str.contains('3'),'amount_new'] = cruce_1_previo['amount']*-1
cruce_1_previo['comparison'] = cruce_1_previo['total_amount_bruto'] == cruce_1_previo['amount_new']
cruce_1_previo.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\659014793.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cruce_1_previo[['transaction_id','status_code','msi']].fillna('NO ENCONTRADO',inplace=True)


,order_id,Type,Date_process_roc,A,Total_amount_charge_roc,Svc_fee_amount_roc,Svc_fee_amount_accel_roc,roc_deposit_date,15,rechazo_amex,...,clave_autorizacion,comision_porcentaje_mdr,comision_porcentaje_msi,comision_porcentaje_final,comision_mdr_db,iva_comision_mdr_db,comision_msi_db,iva_comision_msi_db,amount_new,comparison
0,LMNRTYDPHHMY,4,2024-01-13,376669*****2016****,"95,000.00",0.00,0.00,2024-01-16,0,00,...,010610892240064,-0.02,0.00,-0.02,-22.80,-3.65,0.00,0.00,950.00,True


In [76]:
cruce_1_previo.shape

(1, 35)

In [77]:
cruce_1_previo['status_code'].unique()

array(['1'], dtype=object)

### FILTRAR LOS RECHAZADOS AMEX

In [78]:
cruce_1 = cruce_1_previo[(cruce_1_previo['rechazo_amex'] != '01')]
cruce_1.head()

,order_id,Type,Date_process_roc,A,Total_amount_charge_roc,Svc_fee_amount_roc,Svc_fee_amount_accel_roc,roc_deposit_date,15,rechazo_amex,...,clave_autorizacion,comision_porcentaje_mdr,comision_porcentaje_msi,comision_porcentaje_final,comision_mdr_db,iva_comision_mdr_db,comision_msi_db,iva_comision_msi_db,amount_new,comparison
0,LMNRTYDPHHMY,4,2024-01-13,376669*****2016****,"95,000.00",0.00,0.00,2024-01-16,0,00,...,010610892240064,-0.02,0.00,-0.02,-22.80,-3.65,0.00,0.00,950.00,True


In [79]:
cruce_1.shape

(1, 35)

### RECHAZADOS AMEX

In [80]:
cruce_1_rej = cruce_1_previo[(cruce_1_previo['rechazo_amex'] == '01')]
cruce_1_rej.head()

,order_id,Type,Date_process_roc,A,Total_amount_charge_roc,Svc_fee_amount_roc,Svc_fee_amount_accel_roc,roc_deposit_date,15,rechazo_amex,...,clave_autorizacion,comision_porcentaje_mdr,comision_porcentaje_msi,comision_porcentaje_final,comision_mdr_db,iva_comision_mdr_db,comision_msi_db,iva_comision_msi_db,amount_new,comparison


In [81]:
cruce_1_rej.shape

(0, 35)

# PAYMENTS CRUCE AMEX VS CLIP

In [82]:
txn_epa_deposit = cruce_1[(cruce_1['roc_deposit_date'] >= start_date) & (cruce_1['roc_deposit_date'] <= final_date)]
txn_epa_deposit.head()

,order_id,Type,Date_process_roc,A,Total_amount_charge_roc,Svc_fee_amount_roc,Svc_fee_amount_accel_roc,roc_deposit_date,15,rechazo_amex,...,clave_autorizacion,comision_porcentaje_mdr,comision_porcentaje_msi,comision_porcentaje_final,comision_mdr_db,iva_comision_mdr_db,comision_msi_db,iva_comision_msi_db,amount_new,comparison
0,LMNRTYDPHHMY,4,2024-01-13,376669*****2016****,"95,000.00",0.00,0.00,2024-01-16,0,00,...,010610892240064,-0.02,0.00,-0.02,-22.80,-3.65,0.00,0.00,950.00,True


In [83]:
txn_epa_deposit.shape

(1, 35)

In [84]:
txn_epa_deposit['roc_deposit_date'].unique()

<DatetimeArray>
['2024-01-16 00:00:00']
Length: 1, dtype: datetime64[ns]

In [85]:
duplicates_1 = txn_epa_deposit[txn_epa_deposit.duplicated(['order_id'] , keep=False)]
duplicates_1.sort_values(by=['order_id','Date_process_roc'],inplace=True)
duplicates_1.head()


C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\47295392.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  duplicates_1.sort_values(by=['order_id','Date_process_roc'],inplace=True)


,order_id,Type,Date_process_roc,A,Total_amount_charge_roc,Svc_fee_amount_roc,Svc_fee_amount_accel_roc,roc_deposit_date,15,rechazo_amex,...,clave_autorizacion,comision_porcentaje_mdr,comision_porcentaje_msi,comision_porcentaje_final,comision_mdr_db,iva_comision_mdr_db,comision_msi_db,iva_comision_msi_db,amount_new,comparison


In [86]:
duplicates_1.shape

(0, 35)

In [87]:
# txn_epa_deposit.to_csv('EJEMPLO COMISIONES.csv')

### COMPARISON

In [88]:
comparison_1 = txn_epa_deposit[(txn_epa_deposit['comparison'] == False)]
comparison_1.head()

,order_id,Type,Date_process_roc,A,Total_amount_charge_roc,Svc_fee_amount_roc,Svc_fee_amount_accel_roc,roc_deposit_date,15,rechazo_amex,...,clave_autorizacion,comision_porcentaje_mdr,comision_porcentaje_msi,comision_porcentaje_final,comision_mdr_db,iva_comision_mdr_db,comision_msi_db,iva_comision_msi_db,amount_new,comparison


In [89]:
comparison_1.shape

(0, 35)

### ADJUSTMENTS

In [90]:
txn_epa_adj_trunc = txn_epa_adj[(txn_epa_adj['roc_deposit_date'] >= start_date) &
                                (txn_epa_adj['roc_deposit_date'] <= final_date)]

In [91]:
adj_montos = txn_epa_adj_trunc[(txn_epa_adj_trunc['motivo_adj'] != 'MONTO DE SERVICIO')]
adj_montos.head()

,roc_deposit_date,clave_autorizacion,9,10,A,12,13,no_tarjeta,adj_id,motivo_adj,18,importe_adj_bruto,tasa_adj,importe_impuesto,tasa_servicio_plan_n,importe_adj_neto,bin_bank,last4_bank


In [92]:
adj_montos.shape

(0, 18)

In [93]:
deposit_date_adj = adj_montos.groupby(['roc_deposit_date']).agg({'importe_adj_bruto':sum,'tasa_adj':sum
                                                                 ,'importe_impuesto':sum,'tasa_servicio_plan_n':sum
                                                                 ,'importe_adj_neto':sum}).reset_index()
deposit_date_adj.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\1836096113.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  deposit_date_adj = adj_montos.groupby(['roc_deposit_date']).agg({'importe_adj_bruto':sum,'tasa_adj':sum
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\1836096113.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  deposit_date_adj = adj_montos.groupby(['roc_deposit_date']).agg({'importe_adj_bruto':sum,'tasa_adj':sum
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\1836096113.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupB

,roc_deposit_date,importe_adj_bruto,tasa_adj,importe_impuesto,tasa_servicio_plan_n,importe_adj_neto


In [94]:
deposit_date_adj.shape

(0, 6)

In [95]:
adj_monto_servicio = txn_epa_adj_trunc[(txn_epa_adj_trunc['motivo_adj'] == 'MONTO DE SERVICIO')]
adj_monto_servicio.head()

,roc_deposit_date,clave_autorizacion,9,10,A,12,13,no_tarjeta,adj_id,motivo_adj,18,importe_adj_bruto,tasa_adj,importe_impuesto,tasa_servicio_plan_n,importe_adj_neto,bin_bank,last4_bank


In [96]:
adj_monto_servicio.shape

(0, 18)

In [97]:
pivot_monto_servicio = adj_monto_servicio.groupby(['roc_deposit_date']).agg({'importe_adj_bruto':sum,'tasa_adj':sum
                                                                 ,'importe_impuesto':sum,'tasa_servicio_plan_n':sum
                                                                 ,'importe_adj_neto':sum}).reset_index()
pivot_monto_servicio.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\2826197730.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot_monto_servicio = adj_monto_servicio.groupby(['roc_deposit_date']).agg({'importe_adj_bruto':sum,'tasa_adj':sum
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\2826197730.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot_monto_servicio = adj_monto_servicio.groupby(['roc_deposit_date']).agg({'importe_adj_bruto':sum,'tasa_adj':sum
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\2826197730.py:1: FutureWarning: The provided callable <built-in function sum> is curr

,roc_deposit_date,importe_adj_bruto,tasa_adj,importe_impuesto,tasa_servicio_plan_n,importe_adj_neto


In [98]:
pivot_monto_servicio.shape

(0, 6)

#### ADJUSTMENTS POSITIVOS

In [99]:
adj_montos_positivos = adj_montos[(adj_montos['importe_adj_bruto'] >= 0)]

In [100]:
deposit_date_adj_positivos = adj_montos_positivos.groupby(['roc_deposit_date']).agg({'importe_adj_bruto':sum,'tasa_adj':sum
                                                                 ,'importe_impuesto':sum,'tasa_servicio_plan_n':sum
                                                                 ,'importe_adj_neto':sum}).reset_index()
deposit_date_adj_positivos.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\2047788177.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  deposit_date_adj_positivos = adj_montos_positivos.groupby(['roc_deposit_date']).agg({'importe_adj_bruto':sum,'tasa_adj':sum
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\2047788177.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  deposit_date_adj_positivos = adj_montos_positivos.groupby(['roc_deposit_date']).agg({'importe_adj_bruto':sum,'tasa_adj':sum
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\2047788177.py:1: FutureWarning: The provided callable <built-in funct

,roc_deposit_date,importe_adj_bruto,tasa_adj,importe_impuesto,tasa_servicio_plan_n,importe_adj_neto


In [101]:
deposit_date_adj_positivos.shape

(0, 6)

In [102]:
adj_monto_servicio_positivos = adj_monto_servicio[(adj_monto_servicio['importe_adj_neto'] >= 0)]
adj_monto_servicio_positivos.head()

,roc_deposit_date,clave_autorizacion,9,10,A,12,13,no_tarjeta,adj_id,motivo_adj,18,importe_adj_bruto,tasa_adj,importe_impuesto,tasa_servicio_plan_n,importe_adj_neto,bin_bank,last4_bank


In [103]:
adj_monto_servicio_positivos.shape

(0, 18)

In [104]:
pivot_monto_servicio_positivos = adj_monto_servicio_positivos.groupby(['roc_deposit_date']).agg({'importe_adj_bruto':sum,
                                                                                                 'tasa_adj':sum,
                                                                                                 'importe_impuesto':sum,
                                                                                                 'tasa_servicio_plan_n':sum,
                                                                                                 'importe_adj_neto':sum}).reset_index()
pivot_monto_servicio_positivos.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\670131712.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot_monto_servicio_positivos = adj_monto_servicio_positivos.groupby(['roc_deposit_date']).agg({'importe_adj_bruto':sum,
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\670131712.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot_monto_servicio_positivos = adj_monto_servicio_positivos.groupby(['roc_deposit_date']).agg({'importe_adj_bruto':sum,
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\670131712.py:1: FutureWarning: The provided callable <built-in function sum

,roc_deposit_date,importe_adj_bruto,tasa_adj,importe_impuesto,tasa_servicio_plan_n,importe_adj_neto


In [105]:
pivot_monto_servicio_positivos.shape

(0, 6)

#### ADJUSTMENTS NEGATIVOS

In [106]:
adj_montos_negativos = adj_montos[(adj_montos['importe_adj_bruto'] < 0)]

In [107]:
deposit_date_adj_negativos = adj_montos_negativos.groupby(['roc_deposit_date']).agg({'importe_adj_bruto':sum,
                                                                                     'tasa_adj':sum,
                                                                                     'importe_impuesto':sum,
                                                                                     'tasa_servicio_plan_n':sum,
                                                                                     'importe_adj_neto':sum}).reset_index()
deposit_date_adj_negativos.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\3747470715.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  deposit_date_adj_negativos = adj_montos_negativos.groupby(['roc_deposit_date']).agg({'importe_adj_bruto':sum,
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\3747470715.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  deposit_date_adj_negativos = adj_montos_negativos.groupby(['roc_deposit_date']).agg({'importe_adj_bruto':sum,
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\3747470715.py:1: FutureWarning: The provided callable <built-in function sum> is currently using 

,roc_deposit_date,importe_adj_bruto,tasa_adj,importe_impuesto,tasa_servicio_plan_n,importe_adj_neto


In [108]:
deposit_date_adj_negativos.shape

(0, 6)

In [109]:
adj_monto_servicio_negativos = adj_monto_servicio[(adj_monto_servicio['importe_adj_neto'] < 0)]
adj_monto_servicio_negativos.head()

,roc_deposit_date,clave_autorizacion,9,10,A,12,13,no_tarjeta,adj_id,motivo_adj,18,importe_adj_bruto,tasa_adj,importe_impuesto,tasa_servicio_plan_n,importe_adj_neto,bin_bank,last4_bank


In [110]:
adj_monto_servicio_negativos.shape

(0, 18)

In [111]:
pivot_monto_servicio_negativos = adj_monto_servicio_negativos.groupby(['roc_deposit_date']).agg({'importe_adj_bruto':sum,
                                                                                                 'tasa_adj':sum,
                                                                                                 'importe_impuesto':sum,
                                                                                                 'tasa_servicio_plan_n':sum,
                                                                                                 'importe_adj_neto':sum}).reset_index()
pivot_monto_servicio_negativos.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\1581277042.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot_monto_servicio_negativos = adj_monto_servicio_negativos.groupby(['roc_deposit_date']).agg({'importe_adj_bruto':sum,
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\1581277042.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot_monto_servicio_negativos = adj_monto_servicio_negativos.groupby(['roc_deposit_date']).agg({'importe_adj_bruto':sum,
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\1581277042.py:1: FutureWarning: The provided callable <built-in function 

,roc_deposit_date,importe_adj_bruto,tasa_adj,importe_impuesto,tasa_servicio_plan_n,importe_adj_neto


In [112]:
pivot_monto_servicio_negativos.shape

(0, 6)

### TRANSACCIONES POSITIVAS

In [113]:
deposit_success_txn = txn_epa_deposit[(txn_epa_deposit['total_amount_bruto'] >= 0)]

In [114]:
deposit_success_cruce_1 = deposit_success_txn.groupby(['roc_deposit_date']).agg({'total_amount_bruto':sum,
                                                                                 'fee_amount_mdr':sum,
                                                                                 'fee_amount_msi':sum,
                                                                                 'iva_mdr':sum,
                                                                                 'iva_msi':sum,'comision_mdr_db':sum,
                                                                                 'iva_comision_mdr_db':sum,
                                                                                 'comision_msi_db':sum,
                                                                                 'iva_comision_msi_db':sum}).reset_index()
deposit_success_cruce_1.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\2701766675.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  deposit_success_cruce_1 = deposit_success_txn.groupby(['roc_deposit_date']).agg({'total_amount_bruto':sum,


,roc_deposit_date,total_amount_bruto,fee_amount_mdr,fee_amount_msi,iva_mdr,iva_msi,comision_mdr_db,iva_comision_mdr_db,comision_msi_db,iva_comision_msi_db
0,2024-01-16,950.00,-22.80,0.00,-3.65,0.00,-22.80,-3.65,0.00,0.00


In [115]:
deposit_success_cruce_1.shape

(1, 10)

In [116]:
deposit_success_cruce_2 = deposit_success_cruce_1.merge(deposit_date_adj_positivos[['roc_deposit_date','importe_adj_bruto',
                                                                                    'tasa_adj','importe_impuesto',
                                                                                    'tasa_servicio_plan_n','importe_adj_neto']]
                                                  ,on='roc_deposit_date',how='left')
deposit_success_cruce_2['importe_adj_bruto'].fillna(0,inplace=True)
deposit_success_cruce_2['tasa_adj'].fillna(0,inplace=True)
deposit_success_cruce_2['importe_impuesto'].fillna(0,inplace=True)
deposit_success_cruce_2['tasa_servicio_plan_n'].fillna(0,inplace=True)
deposit_success_cruce_2['importe_adj_neto'].fillna(0,inplace=True)
deposit_success_cruce_2['deposit_amount'] = (deposit_success_cruce_2['total_amount_bruto']
                                             + deposit_success_cruce_2['fee_amount_mdr']
                                             + deposit_success_cruce_2['fee_amount_msi']
                                             + deposit_success_cruce_2['iva_mdr']
                                             + deposit_success_cruce_2['iva_msi']
                                             + deposit_success_cruce_2['importe_adj_bruto']
                                             + deposit_success_cruce_2['tasa_adj']
                                             + deposit_success_cruce_2['importe_impuesto']
                                             + deposit_success_cruce_2['tasa_servicio_plan_n'])
    
deposit_success_cruce_2.head()

,roc_deposit_date,total_amount_bruto,fee_amount_mdr,fee_amount_msi,iva_mdr,iva_msi,comision_mdr_db,iva_comision_mdr_db,comision_msi_db,iva_comision_msi_db,importe_adj_bruto,tasa_adj,importe_impuesto,tasa_servicio_plan_n,importe_adj_neto,deposit_amount
0,2024-01-16,950.00,-22.80,0.00,-3.65,0.00,-22.80,-3.65,0.00,0.00,0.00,0.00,0.00,0.00,0.00,923.55


In [117]:
deposit_success_cruce_2.shape

(1, 16)

In [118]:
deposit_success_cruce_final = deposit_success_cruce_2[['roc_deposit_date','total_amount_bruto','fee_amount_mdr',
                                                       'fee_amount_msi','iva_mdr','iva_msi','comision_mdr_db',
                                                       'iva_comision_mdr_db','comision_msi_db','iva_comision_msi_db',
                                                       'importe_adj_bruto','tasa_adj','importe_impuesto','tasa_servicio_plan_n',
                                                       'deposit_amount']]
deposit_success_cruce_final.head()

,roc_deposit_date,total_amount_bruto,fee_amount_mdr,fee_amount_msi,iva_mdr,iva_msi,comision_mdr_db,iva_comision_mdr_db,comision_msi_db,iva_comision_msi_db,importe_adj_bruto,tasa_adj,importe_impuesto,tasa_servicio_plan_n,deposit_amount
0,2024-01-16,950.00,-22.80,0.00,-3.65,0.00,-22.80,-3.65,0.00,0.00,0.00,0.00,0.00,0.00,923.55


In [119]:
deposit_success_cruce_final.shape

(1, 15)

In [120]:
deposit_success_cruce_final.to_excel('DEPOSITOS POSITIVOS.xlsx')

### CANCELACIONES

In [121]:
deposit_refunded_txn = txn_epa_deposit[(txn_epa_deposit['total_amount_bruto'] < 0)]

In [122]:
deposit_refunded_cruce_1 = deposit_refunded_txn.groupby(['roc_deposit_date']).agg({'total_amount_bruto':sum,
                                                                                   'fee_amount_mdr':sum,
                                                                                   'fee_amount_msi':sum,
                                                                                   'iva_mdr':sum,
                                                                                   'iva_msi':sum,'comision_mdr_db':sum,
                                                                                   'iva_comision_mdr_db':sum,
                                                                                   'comision_msi_db':sum,
                                                                                   'iva_comision_msi_db':sum}).reset_index()
deposit_refunded_cruce_1.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\3762504625.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  deposit_refunded_cruce_1 = deposit_refunded_txn.groupby(['roc_deposit_date']).agg({'total_amount_bruto':sum,
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\3762504625.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  deposit_refunded_cruce_1 = deposit_refunded_txn.groupby(['roc_deposit_date']).agg({'total_amount_bruto':sum,
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\3762504625.py:1: FutureWarning: The provided callable <built-in function sum> is currently using Se

,roc_deposit_date,total_amount_bruto,fee_amount_mdr,fee_amount_msi,iva_mdr,iva_msi,comision_mdr_db,iva_comision_mdr_db,comision_msi_db,iva_comision_msi_db


In [123]:
deposit_refunded_cruce_1.shape

(0, 10)

In [124]:
deposit_refunded_cruce_2 = deposit_refunded_cruce_1.merge(deposit_date_adj_negativos[['roc_deposit_date','importe_adj_bruto',
                                                                                      'tasa_adj','importe_impuesto',
                                                                                      'tasa_servicio_plan_n',
                                                                                      'importe_adj_neto']],
                                                          on='roc_deposit_date',how='right')
deposit_refunded_cruce_2.sort_values(by=['roc_deposit_date'],inplace=True)
deposit_refunded_cruce_2['total_amount_bruto'].fillna(0,inplace=True)
deposit_refunded_cruce_2['fee_amount_mdr'].fillna(0,inplace=True)
deposit_refunded_cruce_2['fee_amount_msi'].fillna(0,inplace=True)
deposit_refunded_cruce_2['iva_mdr'].fillna(0,inplace=True)
deposit_refunded_cruce_2['iva_msi'].fillna(0,inplace=True)
deposit_refunded_cruce_2['importe_adj_bruto'].fillna(0,inplace=True)
deposit_refunded_cruce_2['tasa_adj'].fillna(0,inplace=True)
deposit_refunded_cruce_2['importe_impuesto'].fillna(0,inplace=True)
deposit_refunded_cruce_2['tasa_servicio_plan_n'].fillna(0,inplace=True)
deposit_refunded_cruce_2['importe_adj_neto'].fillna(0,inplace=True)
deposit_refunded_cruce_2['comision_mdr_db'].fillna(0,inplace=True)
deposit_refunded_cruce_2['iva_comision_mdr_db'].fillna(0,inplace=True)
deposit_refunded_cruce_2['comision_msi_db'].fillna(0,inplace=True)
deposit_refunded_cruce_2['iva_comision_mdr_db'].fillna(0,inplace=True)
deposit_refunded_cruce_2['deposit_amount'] = (deposit_refunded_cruce_2['total_amount_bruto']
                                             + deposit_refunded_cruce_2['fee_amount_mdr']
                                             + deposit_refunded_cruce_2['fee_amount_msi']
                                             + deposit_refunded_cruce_2['iva_mdr']
                                             + deposit_refunded_cruce_2['iva_msi']
                                             + deposit_refunded_cruce_2['importe_adj_bruto']
                                             + deposit_refunded_cruce_2['tasa_adj']
                                             + deposit_refunded_cruce_2['importe_impuesto']
                                             + deposit_refunded_cruce_2['tasa_servicio_plan_n'])
    
deposit_refunded_cruce_2.head()

,total_amount_bruto,fee_amount_mdr,fee_amount_msi,iva_mdr,iva_msi,comision_mdr_db,iva_comision_mdr_db,comision_msi_db,iva_comision_msi_db,roc_deposit_date,importe_adj_bruto,tasa_adj,importe_impuesto,tasa_servicio_plan_n,importe_adj_neto,deposit_amount


In [125]:
deposit_refunded_cruce_2.shape

(0, 16)

In [126]:
deposit_refunded_cruce_final = deposit_refunded_cruce_2[['roc_deposit_date','total_amount_bruto','fee_amount_mdr',
                                                         'fee_amount_msi','iva_mdr','iva_msi','comision_mdr_db',
                                                         'iva_comision_mdr_db','comision_msi_db','iva_comision_msi_db',
                                                         'importe_adj_bruto','tasa_adj','importe_impuesto',
                                                         'tasa_servicio_plan_n','deposit_amount']]
deposit_refunded_cruce_final.head()

,roc_deposit_date,total_amount_bruto,fee_amount_mdr,fee_amount_msi,iva_mdr,iva_msi,comision_mdr_db,iva_comision_mdr_db,comision_msi_db,iva_comision_msi_db,importe_adj_bruto,tasa_adj,importe_impuesto,tasa_servicio_plan_n,deposit_amount


In [127]:
deposit_refunded_cruce_final.shape

(0, 15)

In [128]:
deposit_refunded_cruce_final.to_excel('DEPOSITOS NEGATIVOS.xlsx')

### DEPOSITOS FINALES

In [129]:
deposit_date_cruce_1 = txn_epa_deposit.groupby(['roc_deposit_date']).agg({'total_amount_bruto':sum,
                                                                          'fee_amount_mdr':sum,
                                                                          'fee_amount_msi':sum,
                                                                          'iva_mdr':sum,
                                                                          'iva_msi':sum,
                                                                          'comision_mdr_db':sum,
                                                                          'iva_comision_mdr_db':sum,
                                                                          'comision_msi_db':sum,
                                                                          'iva_comision_msi_db':sum}).reset_index()
deposit_date_cruce_1.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\3481331134.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  deposit_date_cruce_1 = txn_epa_deposit.groupby(['roc_deposit_date']).agg({'total_amount_bruto':sum,


,roc_deposit_date,total_amount_bruto,fee_amount_mdr,fee_amount_msi,iva_mdr,iva_msi,comision_mdr_db,iva_comision_mdr_db,comision_msi_db,iva_comision_msi_db
0,2024-01-16,950.00,-22.80,0.00,-3.65,0.00,-22.80,-3.65,0.00,0.00


In [130]:
deposit_date_cruce_1.shape

(1, 10)

In [131]:
deposit_date_cruce_2 = deposit_date_cruce_1.merge(deposit_date_adj[['roc_deposit_date','importe_adj_bruto','tasa_adj',
                                                                    'importe_impuesto','tasa_servicio_plan_n',
                                                                    'importe_adj_neto']],
                                                  on='roc_deposit_date',how='left')
deposit_date_cruce_2['importe_adj_bruto'].fillna(0,inplace=True)
deposit_date_cruce_2['tasa_adj'].fillna(0,inplace=True)
deposit_date_cruce_2['importe_impuesto'].fillna(0,inplace=True)
deposit_date_cruce_2['tasa_servicio_plan_n'].fillna(0,inplace=True)
deposit_date_cruce_2['importe_adj_neto'].fillna(0,inplace=True)
deposit_date_cruce_2['deposit_amount'] = (deposit_date_cruce_2['total_amount_bruto'] + deposit_date_cruce_2['fee_amount_mdr']
                                          + deposit_date_cruce_2['fee_amount_msi'] + deposit_date_cruce_2['iva_mdr']
                                          + deposit_date_cruce_2['iva_msi'] + deposit_date_cruce_2['importe_adj_bruto']
                                          + deposit_date_cruce_2['tasa_adj'] + deposit_date_cruce_2['importe_impuesto']
                                          + deposit_date_cruce_2['tasa_servicio_plan_n'])
    
deposit_date_cruce_2.head()

,roc_deposit_date,total_amount_bruto,fee_amount_mdr,fee_amount_msi,iva_mdr,iva_msi,comision_mdr_db,iva_comision_mdr_db,comision_msi_db,iva_comision_msi_db,importe_adj_bruto,tasa_adj,importe_impuesto,tasa_servicio_plan_n,importe_adj_neto,deposit_amount
0,2024-01-16,950.00,-22.80,0.00,-3.65,0.00,-22.80,-3.65,0.00,0.00,0.00,0.00,0.00,0.00,0.00,923.55


In [132]:
deposit_date_cruce_final = deposit_date_cruce_2[['roc_deposit_date','total_amount_bruto','fee_amount_mdr','fee_amount_msi',
                                                 'iva_mdr','iva_msi','comision_mdr_db','iva_comision_mdr_db','comision_msi_db',
                                                 'iva_comision_msi_db','importe_adj_bruto','tasa_adj','importe_impuesto',
                                                 'tasa_servicio_plan_n','deposit_amount']]
deposit_date_cruce_final.head()

,roc_deposit_date,total_amount_bruto,fee_amount_mdr,fee_amount_msi,iva_mdr,iva_msi,comision_mdr_db,iva_comision_mdr_db,comision_msi_db,iva_comision_msi_db,importe_adj_bruto,tasa_adj,importe_impuesto,tasa_servicio_plan_n,deposit_amount
0,2024-01-16,950.00,-22.80,0.00,-3.65,0.00,-22.80,-3.65,0.00,0.00,0.00,0.00,0.00,0.00,923.55


In [133]:
deposit_date_cruce_final.shape

(1, 15)

# TRANSACTIONS AMEX - CLIP

In [134]:
txn_epa_txn = cruce_1[(cruce_1['Date_process_roc'] >= start_date) & (cruce_1['Date_process_roc'] <= final_date)]
txn_epa_txn.sort_values(by=['Date_process_roc','order_id'],inplace=True)
txn_epa_txn.head()

,order_id,Type,Date_process_roc,A,Total_amount_charge_roc,Svc_fee_amount_roc,Svc_fee_amount_accel_roc,roc_deposit_date,15,rechazo_amex,...,clave_autorizacion,comision_porcentaje_mdr,comision_porcentaje_msi,comision_porcentaje_final,comision_mdr_db,iva_comision_mdr_db,comision_msi_db,iva_comision_msi_db,amount_new,comparison
0,LMNRTYDPHHMY,4,2024-01-13,376669*****2016****,"95,000.00",0.00,0.00,2024-01-16,0,00,...,010610892240064,-0.02,0.00,-0.02,-22.80,-3.65,0.00,0.00,950.00,True


In [135]:
txn_epa_txn.shape

(1, 35)

In [136]:
duplicates_2 = txn_epa_txn[txn_epa_txn.duplicated(['order_id'] , keep=False)]
duplicates_2.sort_values(by=['order_id','Date_process_roc'],inplace=True)
duplicates_2.head()


C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\1014910255.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  duplicates_2.sort_values(by=['order_id','Date_process_roc'],inplace=True)


,order_id,Type,Date_process_roc,A,Total_amount_charge_roc,Svc_fee_amount_roc,Svc_fee_amount_accel_roc,roc_deposit_date,15,rechazo_amex,...,clave_autorizacion,comision_porcentaje_mdr,comision_porcentaje_msi,comision_porcentaje_final,comision_mdr_db,iva_comision_mdr_db,comision_msi_db,iva_comision_msi_db,amount_new,comparison


In [137]:
duplicates_2.shape

(0, 35)

In [138]:
txn_epa_txn.drop_duplicates(subset ='order_id', keep='last', inplace=True)
txn_epa_txn.shape

(1, 35)

In [139]:
comparison_2 = txn_epa_txn[(txn_epa_txn['comparison'] == False)]
comparison_2.head()

,order_id,Type,Date_process_roc,A,Total_amount_charge_roc,Svc_fee_amount_roc,Svc_fee_amount_accel_roc,roc_deposit_date,15,rechazo_amex,...,clave_autorizacion,comision_porcentaje_mdr,comision_porcentaje_msi,comision_porcentaje_final,comision_mdr_db,iva_comision_mdr_db,comision_msi_db,iva_comision_msi_db,amount_new,comparison


In [140]:
comparison_2.shape

(0, 35)

# GROUP BY TRANSACTION DATE

In [141]:
txn_txn_date_cruce_1 = txn_epa_txn.groupby(['Date_process_roc']).agg({'total_amount_bruto':sum,                                                                      'fee_amount_mdr':sum,
                                                                      'fee_amount_msi':sum,
                                                                      'iva_mdr':sum,
                                                                      'iva_msi':sum,
                                                                      'comision_mdr_db':sum,
                                                                      'iva_comision_mdr_db':sum,
                                                                      'comision_msi_db':sum,
                                                                      'iva_comision_msi_db':sum,
                                                                      'amount_new':sum,
                                                                      'order_id':'count'
}).reset_index()
# txn_txn_date_cruce_1.rename(columns ={'order_id':'number_transactions'}, inplace=True)
# txn_txn_date_cruce_1['diff'] = txn_txn_date_cruce_1['total_amount_bruto'] - txn_txn_date_cruce_1['amount_new']
txn_txn_date_cruce_1.head()


C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\3380768711.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  txn_txn_date_cruce_1 = txn_epa_txn.groupby(['Date_process_roc']).agg({'total_amount_bruto':sum,                                                                      'fee_amount_mdr':sum,


,Date_process_roc,total_amount_bruto,fee_amount_mdr,fee_amount_msi,iva_mdr,iva_msi,comision_mdr_db,iva_comision_mdr_db,comision_msi_db,iva_comision_msi_db,amount_new,order_id
0,2024-01-13,950.00,-22.80,0.00,-3.65,0.00,-22.80,-3.65,0.00,0.00,950.00,1


In [142]:
txn_txn_date_cruce_1.shape

(1, 12)

# GROUP BY TRANSACTION DATE AND STATUS CODE

In [143]:
txn_txn_status_cruce_1 = txn_epa_txn.groupby(['Date_process_roc','status_code']).agg({'total_amount_bruto':sum,
                                                                                      'fee_amount_mdr':sum,
                                                                                      'fee_amount_msi':sum,
                                                                                      'iva_mdr':sum,
                                                                                      'iva_msi':sum,
                                                                                      'comision_mdr_db':sum,
                                                                                      'iva_comision_mdr_db':sum,
                                                                                      'comision_msi_db':sum,
                                                                                      'iva_comision_msi_db':sum,
                                                                                      'amount_new':sum,
                                                                                      'order_id':'count'}).reset_index()
txn_txn_status_cruce_1.rename(columns ={'order_id':'number_transactions'}, inplace=True)
txn_txn_status_cruce_1['diff'] = txn_txn_status_cruce_1['total_amount_bruto'] - txn_txn_status_cruce_1['amount_new']
txn_txn_status_cruce_1.head()


C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\1814311225.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  txn_txn_status_cruce_1 = txn_epa_txn.groupby(['Date_process_roc','status_code']).agg({'total_amount_bruto':sum,
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\1814311225.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  txn_txn_status_cruce_1 = txn_epa_txn.groupby(['Date_process_roc','status_code']).agg({'total_amount_bruto':sum,


,Date_process_roc,status_code,total_amount_bruto,fee_amount_mdr,fee_amount_msi,iva_mdr,iva_msi,comision_mdr_db,iva_comision_mdr_db,comision_msi_db,iva_comision_msi_db,amount_new,number_transactions,diff
0,2024-01-13,1,950.00,-22.80,0.00,-3.65,0.00,-22.80,-3.65,0.00,0.00,950.00,1,0.00


In [144]:
txn_txn_status_cruce_1.dtypes

Date_process_roc       datetime64[ns]
status_code                    object
total_amount_bruto            float64
fee_amount_mdr                float64
fee_amount_msi                float64
iva_mdr                       float64
iva_msi                       float64
comision_mdr_db               float64
iva_comision_mdr_db           float64
comision_msi_db               float64
iva_comision_msi_db           float64
amount_new                    float64
number_transactions             int64
diff                          float64
dtype: object

In [145]:
txn_txn_status_cruce_1.shape

(1, 14)

### STATUS 1

In [146]:
txn_txn_status_cruce_1_status_1 = txn_txn_status_cruce_1[(txn_txn_status_cruce_1['status_code'] == '1')]
txn_txn_status_cruce_1_status_1.head()

,Date_process_roc,status_code,total_amount_bruto,fee_amount_mdr,fee_amount_msi,iva_mdr,iva_msi,comision_mdr_db,iva_comision_mdr_db,comision_msi_db,iva_comision_msi_db,amount_new,number_transactions,diff
0,2024-01-13,1,950.00,-22.80,0.00,-3.65,0.00,-22.80,-3.65,0.00,0.00,950.00,1,0.00


In [147]:
txn_txn_status_cruce_1_status_1.shape

(1, 14)

### STATUS 3

In [148]:
txn_txn_status_cruce_1_status_3 = txn_txn_status_cruce_1[(txn_txn_status_cruce_1['status_code'] == '3')]
txn_txn_status_cruce_1_status_3.head()

,Date_process_roc,status_code,total_amount_bruto,fee_amount_mdr,fee_amount_msi,iva_mdr,iva_msi,comision_mdr_db,iva_comision_mdr_db,comision_msi_db,iva_comision_msi_db,amount_new,number_transactions,diff


In [149]:
txn_txn_status_cruce_1_status_3.shape

(0, 14)

# POSIBLES DISPUTAS

In [150]:
posibles_disputas = txn_epa_txn[(txn_epa_txn['status_code'] != '1') & (txn_epa_txn['status_code'] != '3')]
posibles_disputas.head()


,order_id,Type,Date_process_roc,A,Total_amount_charge_roc,Svc_fee_amount_roc,Svc_fee_amount_accel_roc,roc_deposit_date,15,rechazo_amex,...,clave_autorizacion,comision_porcentaje_mdr,comision_porcentaje_msi,comision_porcentaje_final,comision_mdr_db,iva_comision_mdr_db,comision_msi_db,iva_comision_msi_db,amount_new,comparison


In [151]:
posibles_disputas.shape

(0, 35)

### POSIBLES DISPUTAS BY DATE

In [152]:
txn_txn_status_cruce_1_posibles_disputas = posibles_disputas.groupby(['Date_process_roc']).agg({'total_amount_bruto':sum,
                                                                                                'fee_amount_mdr':sum,
                                                                                                'fee_amount_msi':sum,
                                                                                                'iva_mdr':sum,'iva_msi':sum,
                                                                                                'amount_new':sum,
                                                                                                'comision_mdr_db':sum,
                                                                                                'iva_comision_mdr_db':sum,
                                                                                                'comision_msi_db':sum,
                                                                                                'iva_comision_msi_db':sum,
                                                                                                'order_id':'count'}).reset_index()
txn_txn_status_cruce_1_posibles_disputas.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\3403979889.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  txn_txn_status_cruce_1_posibles_disputas = posibles_disputas.groupby(['Date_process_roc']).agg({'total_amount_bruto':sum,
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\3403979889.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  txn_txn_status_cruce_1_posibles_disputas = posibles_disputas.groupby(['Date_process_roc']).agg({'total_amount_bruto':sum,
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_1404\3403979889.py:1: FutureWarning: The provided callable <built-in function 

,Date_process_roc,total_amount_bruto,fee_amount_mdr,fee_amount_msi,iva_mdr,iva_msi,amount_new,comision_mdr_db,iva_comision_mdr_db,comision_msi_db,iva_comision_msi_db,order_id


In [153]:
txn_txn_status_cruce_1_posibles_disputas.shape

(0, 12)

# EXCEL

## PAYOUTS

In [154]:
writer = pd.ExcelWriter('EPA PAYOUTS ENERO CON COMISIONES 2024.xlsx',
                        engine='xlsxwriter',
                        engine_kwargs={'options': {'encoding':'utf-8'}})
deposit_date_cruce_final.to_excel(writer, sheet_name='PAYOUTS',index=False)
deposit_success_cruce_1.to_excel(writer, sheet_name='PAYOUTS SUCCESS TXN',index=False)
deposit_refunded_cruce_1.to_excel(writer, sheet_name='PAYOUTS REFUNDED TXN',index=False)
epa_1_excel_final.to_excel(writer, sheet_name='EPA PAYMENT RECORD',index=False)
epa_3_excel_final.to_excel(writer, sheet_name='EPA SUM OF CHARGE',index=False)
txn_epa_deposit.to_excel(writer, sheet_name='PAYOUTS DETAIL',index=False)
cruce_1_rej.to_excel(writer, sheet_name='RECHAZOS AMEX',index=False)
duplicates_1.to_excel(writer, sheet_name='PAYOUTS DUPLICATES',index=False)
comparison_1.to_excel(writer, sheet_name='PAYOUTS COMPARISON',index=False)
txn_epa_adj_final.to_excel(writer, sheet_name='ADJUSTMENS DETAIL',index=False)
deposit_date_adj_positivos.to_excel(writer, sheet_name='ADJUSTMENTS POSITIVOS',index=False)
pivot_monto_servicio_positivos.to_excel(writer, sheet_name='MONTO SERVICIO POSIVOS',index=False)
deposit_date_adj_negativos.to_excel(writer, sheet_name='ADJUSTMENTS NEGATIVOS',index=False)
pivot_monto_servicio_negativos.to_excel(writer, sheet_name='MONTO SERVICIO NEGATIVOS',index=False)
writer.close()

## TRANSACCIONES

In [155]:
writer = pd.ExcelWriter('EPA TRANSACCIONES CON COMISIONES ENERO 2024.xlsx',
                        engine='xlsxwriter',
                        engine_kwargs={'options': {'encoding':'utf-8'}})
txn_epa_txn.to_excel(writer, sheet_name='TRANSACTIONS DETAIL',index=False)
cruce_1_rej.to_excel(writer, sheet_name='RECHAZOS AMEX',index=False)
duplicates_2.to_excel(writer, sheet_name='TRANSACTIONS DUPLICATES',index=False)
comparison_2.to_excel(writer, sheet_name='TRANSACTIONS COMPARISON',index=False)
txn_txn_date_cruce_1.to_excel(writer, sheet_name='TXN DATE',index=False)
txn_txn_status_cruce_1.to_excel(writer, sheet_name='TXN DATE AND STATUS',index=False)
txn_txn_status_cruce_1_status_1.to_excel(writer, sheet_name='STATUS 1',index=False)
txn_txn_status_cruce_1_status_3.to_excel(writer, sheet_name='STATUS 3',index=False)
txn_txn_status_cruce_1_posibles_disputas.to_excel(writer, sheet_name='POSIBLES DISPUTAS BY DATE',index=False)
posibles_disputas.to_excel(writer, sheet_name='POSIBLES DISPUTAS',index=False)
writer.close()